In [1]:
import pandas as pd
import numpy as np

crm = pd.read_excel(r'C:\Users\Mironov1-AM\PycharmProjects\Офис\Gifts\CRM_cut.xlsx')
tmc = pd.read_excel(r'C:\Users\Mironov1-AM\PycharmProjects\Офис\Gifts\TMC.xlsx')

crm['GIFT_NAME1'] = crm['GIFT_NAME1'].str.lower()
crm['GIFT_NAME1'] = crm['GIFT_NAME1'].str.strip()

tmc['find_text'] = tmc['find_text'].str.lower()
tmc['find_text'] = tmc['find_text'].str.strip()
tmc['exclude_text'] = tmc['exclude_text'].str.lower()
tmc['exclude_text'] = tmc['exclude_text'].str.strip()

stop_words = [',', '.']

for word in stop_words:
    crm['GIFT_NAME1'] = crm['GIFT_NAME1'].str.replace(word, '', regex=False)
    tmc['find_text'] = tmc['find_text'].str.replace(word, '',  regex=False)


In [2]:
def func_find_tmc(x: dict):
    id_tmc = -1

    if type(x['GIFT_SUM1']) == str or pd.isnull(x['GIFT_SUM1']) or pd.isnull(x['GIFT_NAME1']):
        return id_tmc

    crm_text = x['GIFT_NAME1']
    crm_price = x['GIFT_SUM1']

    tmp = pd.DataFrame(columns=[['price', 'short_name']])   #### Исправлено
    tmp = [{'price': 1, 'short_name': "f"}] ####!!!!!!!!!!колхоз
    tmp = pd.DataFrame(tmp) ####!!!!!!!!!!колхоз
    tmp = tmp.drop(index=[0])  ####!!!!!!!!!!колхоз

    for i in range(tmc.shape[0]):
        is_find = True
        find_text = tmc.loc[i, 'find_text'].split()
        for text in find_text:
            if text not in crm_text:
                is_find = False
                break
        if not pd.isnull(tmc.loc[i, 'exclude_text']):
            exclude_text = tmc.loc[i, 'exclude_text'].split()
            for text in exclude_text:
                if text in crm_text:
                    is_find = False
                    break

        if not is_find:
            continue

        tmp = tmp.append(tmc.loc[i, ['price', 'short_name']], ignore_index=True)
    tmp['ones'] = 1
    tmp = tmp.reset_index()

    merge_tmc = list(tmp['short_name'].unique())
    merged_df = tmp[tmp['short_name'] == merge_tmc[0]].reset_index()

    for j, word in enumerate(merge_tmc[1:]):
        merged_df = merged_df.merge(
            tmp[tmp['short_name'] == word],
            on='ones',
            how='outer',
            suffixes=['_' + str(j), '_' + str(j + 1)]
        )

    merged_df['total_cost'] = 0
    merged_df['all_tmc'] = ''
    for column in merged_df.columns:
        if 'price' in column:
            merged_df['total_cost'] += merged_df[column]
        if 'short_name' in column:
            merged_df['all_tmc'] += ' ' + merged_df[column]

    merged_df['different_price'] = abs(merged_df['total_cost'] - crm_price)
    result = merged_df.sort_values('different_price').reset_index(drop=True)

    if len(result) > 0:
        result = result.loc[0, ['all_tmc']]
    else:
        return result
    result = list(result)
    
    return result   #### (100, 'блокнот шоколад ручка')  То что должно выходить
fun = func_find_tmc({'GIFT_SUM1': 500, 'GIFT_NAME1': 'блокнот шоколад ручка'})
fun

[' Блокнот Шоколад Ручка']

In [3]:

# fun.to_excel('resultTEST.xlsx', index=False)
# crm['all_tmc'] = crm.apply(func_find_tmc, axis=1)

In [4]:
type(fun)

list

In [5]:
tmp = pd.DataFrame(columns=[['price', 'short_name']])
tmp = [{'price': 1, 'short_name': "f"}]
tmp = pd.DataFrame(tmp) 
tmp = tmp.drop(index=[0])   
tmp = tmp.append(tmc.loc[7, ['price', 'short_name']], ignore_index=True)
tmp = tmp.append(tmc.loc[3, ['price', 'short_name']], ignore_index=True) #добавлена для пробы, в код не вносить!
tmp['ones'] = 1
tmp = tmp.reset_index()
tmp

,index,price,short_name,ones
0,0,546.0,Блокнот,1
1,1,112.8,Бандана,1


In [6]:
merged_df = pd.DataFrame(columns=tmp.columns)

merged_df = [{'index': 0, 'price': "8", 'short_name': "f", 'ones': "1", 'total_price': "12", 'all_tmc': "sd"}] 
merged_df = pd.DataFrame(merged_df) 
merged_df = merged_df.drop(index=[0])
merged_df

,index,price,short_name,ones,total_price,all_tmc


In [7]:
for j, word in enumerate(tmp['short_name'].unique()):
        merged_df = merged_df.merge(
            tmp[tmp['short_name'] == word],
            on='ones',
            how='outer',
            suffixes=['_' + str(j), '_' + str(j + 1)]
        )
merged_df

,index_0,price_0,short_name_0,total_price,all_tmc,index_1,price_1,short_name_1,ones,index,price,short_name
0,NaN,NaN,NaN,NaN,NaN,0,546.0,Блокнот,1,1,112.8,Бандана


In [8]:
merged_df['total_price'] = 0
merged_df['all_tmc'] = ''
for column in merged_df.columns:
    if 'price' in column:
        merged_df['total_price'] += merged_df[column]
    if 'short_name' in column:
        merged_df['all_tmc'] += ' ' + merged_df[column]
        print(merged_df)

   index_0 price_0 short_name_0 total_price all_tmc  index_1  price_1  \
0      NaN     NaN          NaN         NaN     NaN        0    546.0   

  short_name_1  ones  index  price short_name  
0      Блокнот     1      1  112.8    Бандана  
   index_0 price_0 short_name_0 total_price all_tmc  index_1  price_1  \
0      NaN     NaN          NaN         NaN     NaN        0    546.0   

  short_name_1  ones  index  price short_name  
0      Блокнот     1      1  112.8    Бандана  
   index_0 price_0 short_name_0 total_price all_tmc  index_1  price_1  \
0      NaN     NaN          NaN         NaN     NaN        0    546.0   

  short_name_1  ones  index  price short_name  
0      Блокнот     1      1  112.8    Бандана  


In [9]:
# result = result['all_tms'][1:]

In [14]:
a = merged_df['short_name_1'][0]
a[1:]

'локнот'

In [ ]:
    for i in range(tmc.shape[0]):
        is_find = True
        find_text = tmc.loc[i, 'find_text'].split()
        for text in find_text:
            if text not in y:
                if not is_levenshtein:
                    is_find = False
                    break
                for crm_word in crm_words:
                    leven = levenshtein(crm_word, text)
                    if leven <= 1:
                        is_find = True
                        break
                    else:
                        is_find = False
        if not pd.isnull(tmc.loc[i, 'exclude_text']):
            exclude_text = tmc.loc[i, 'exclude_text'].split()
            for text in exclude_text:
                if text in y:
                    is_find = False
                    break

        if not is_find:
            continue